In [1]:
import sys

sys.path.append("..")

import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import time

from methods.kth.epssample import *
from methods.kth.fagin import *
from methods.kth.threshold import *
from methods.utils import *
from methods.range_search.hierarchy import *
from ranges.stripe_range import StripeRange

Utils

In [2]:
def index_fn(points):
    ind = HierarchicalIndex(points, decay=4)
    ind.build_index()
    ind.find_coverage()
    ind.find_neighbor_stats()
    return ind

Time vs different k:

In [3]:
# ns = [10000, 50000, 100000, 200000]
# dim = 4
# ks = [1 / 2, 1 / 4, 1 / 8, 1 / 16, 1 / 32, 1 / 64]

# eps = 1 / 32

# results = {}

# for n in ns:
#     print(f"Running for n={n} / {ns}...")
#     points = np.random.zipf(1.5, size=(n, dim))
#     weights = np.random.rand(dim)
#     weights /= np.linalg.norm(weights)

#     print("Building index...")
#     prep = preprocess(points, index_fn=index_fn, eps=eps)

#     for l in ks:
#         k = int(l * n)
#         print(f"Running for k={k} / {ks}...")
#         start = time.time()
#         for _ in range(5):
#             res = find_kth(*prep, weights=weights, k=k)
#         end = time.time()
#         results.setdefault("our_time", []).append((end - start) / 5)

#         start = time.time()
#         for _ in range(5):
#             gt = find_kth_exhaustive(points, weights, k=k)
#         end = time.time()
#         results.setdefault("exhaustive_time", []).append((end - start) / 5)

#         print("gt", gt)
#         print("res", res)
#         print(points[res])
#         if gt[0] in points[res]:
#             results.setdefault("correct", []).append(1)
#         else:
#             results.setdefault("correct", []).append(0)

#         results.setdefault("k", []).append(k)
#         results.setdefault("n", []).append(n)
#         results.setdefault("dim", []).append(dim)

visualize

In [ ]:
ns = [10000]
dim = 16
ks = [1 / 2, 1 / 4, 1 / 8, 1 / 16, 1 / 32, 1 / 64]

eps = 1 / 20

results = {}

for n in ns:
    print(f"Running for n={n} / {ns}...")
    points = np.random.zipf(1.5, size=(n, dim))
    weights = np.random.rand(dim)
    weights /= np.linalg.norm(weights)

    print("Building index...")
    prep = preprocess(points, index_fn=index_fn, eps=eps)

    for l in ks:
        k = int(l * n)
        print(f"Running for k={k} / {ks}...")
        start = time.time()
        # for _ in range(1):
        # res = find_kth(*prep, weights=weights, k=k)
        stripe = find_stripe(prep[1], prep[2], weights, k, prep[3])
        end_t = time.time()
        res = query(prep[0], stripe)
        end = time.time()
        results.setdefault("our_time_2", []).append((end - end_t) / 5)
        results.setdefault("our_time_1", []).append((end_t - start) / 5)

        start = time.time()
        for _ in range(1):
            gt = find_kth_exhaustive(points, weights, k=k)
        end = time.time()
        results.setdefault("exhaustive_time", []).append((end - start) / 5)

        # print("gt", gt)
        # print("res", res)
        # print(points[res])
        if gt[0] in points[res]:
            results.setdefault("correct", []).append(1)
            print("correct")
        else:
            results.setdefault("correct", []).append(0)
            print("incorrect")

        results.setdefault("k", []).append(k)
        results.setdefault("n", []).append(n)
        results.setdefault("dim", []).append(dim)

np.mean(results["our_time_1"]), np.mean(results["our_time_2"]), np.mean(
    results["exhaustive_time"]
), np.mean(results["correct"])

Running for n=10000 / [10000]...
Building index...
L: 7
